In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
from bs4 import BeautifulSoup # documentation available at : www.crummy.com/software/BeautifulSoup/bs4/doc/
import requests # sends http requests and access the page : docs.python-requests.org/en/latest/
import csv # creates the output csv file
from bs4 import BeautifulSoup, Tag
#import unicodedata # works with string encoding of the data

In [2]:
entries = []
entry = []
urlnumber = 1 # Give the page number to start with

In [3]:

while urlnumber < 544:  # Set the last page number you want to scrape
    url = f'https://forums.edmunds.com/discussion/7526/general/x/midsize-sedans-2-0/p{urlnumber}'

    try:
        r = requests.get(url, timeout=10)  # Sending a request to access the page
        r.raise_for_status()  # This will raise an HTTPError if the HTTP request returned an unsuccessful status code
    except Exception as e:
        print("Error message:", e)
        break

    data = r.text
    soup = BeautifulSoup(data, 'lxml')  # Parsing the page source with lxml parser

    for outer_div in soup.find_all('div', class_='Comment'):
        entry = []
        try:
            # Skip entries with embedded media like YouTube videos
            if outer_div.find('iframe', src=lambda value: 'youtube' in value if value else False):
                continue

            # Skip entries with images or other embedded content that is not text
            if outer_div.find(['img', 'video', 'embed', 'object']):
                continue

            # Process the author of the comment
            author_info = outer_div.find('a', class_='Username')
            if author_info and author_info.string:
                entry.append(author_info.string.strip().encode("utf-8"))

            # Process the date and time of the comment
            time_info = outer_div.find('time')
            if time_info and time_info.string:
                entry.append(time_info.string.strip().encode('iso-8859-1'))

            # Process the main message of the comment
            message_div = outer_div.find('div', class_='Message')
            if message_div:
                # Remove non-text elements like emojis or other media
                for non_text in message_div.find_all(['img', 'video', 'embed', 'object']):
                    non_text.decompose()
                
                # Get the text content, replacing line breaks with spaces
                text = message_div.get_text(separator=" ", strip=True)
                entry.append(text.encode('ascii', 'replace'))

            # If the entry has content, add it to the entries list
            if entry:
                entries.append(entry)

        except Exception as inner_e:
            print("Error processing a comment:", inner_e)
            continue  # Skip this comment and continue with the next one

    urlnumber += 1  # Increment the page number to move to the next page

# Output or process your entries list as needed


In [4]:
# Convert a list of byte to list a of string     
stringlist=[[x.decode('iso-8859-1') for x in entry] for entry in entries]
# Save the list to a csv file

headers = ['date', 'userid', 'message']

with open('edmunds_extraction.csv', 'w') as output:
    writer = csv.writer(output, quoting=csv.QUOTE_ALL)
    writer.writerow(headers)
    writer.writerows(stringlist)

print ("Wrote to edmunds_extraction.csv")

Wrote to edmunds_extraction.csv
